In [2]:
import pymongo
import pandas as pd
import numpy as np
import re
import time
import datetime
import nltk
from datetime import datetime, timedelta
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
client = pymongo.MongoClient('localhost', 27017)
db = client.db
threads = db.threads
companies = db.companylist

In [72]:
def getContinuousChunks(text):
    chunked = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text)))
    continuousChunk = []
    currentChunk = []
    for i in chunked:
        if type(i) == nltk.tree.Tree and i.label() == 'ORGANIZATION':
            currentChunk.append(" ".join([token for token, pos in i.leaves()]))
        if currentChunk:
            namedEntity = " ".join(currentChunk)
            if namedEntity not in continuousChunk:
                continuousChunk.append(namedEntity)
                currentChunk = []
        else:
            continue
    return continuousChunk
def ngrams(string, n=2):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

def mapResults(results, leftNames, rightNames):
    matchdf = pd.DataFrame(0, index = np.arange(len(result.nonzero()[0])), columns = ['Left', 'Right', 'Similarity'])
    for i in range(len(result.nonzero()[0])):
        matchdf.loc[i, 'Left'] = leftNames[result.nonzero()[0][i]]
        matchdf.loc[i, 'Right'] = rightNames[result.nonzero()[1][i]]
        matchdf.loc[i, 'Similarity'] = result[result.nonzero()[0][i]][result.nonzero()[1][i]]
    return matchdf

In [82]:
df = pd.DataFrame.from_records(threads.find({'Label' :{'$ne': 0}}))
companydf = pd.DataFrame.from_records(companies.find())

In [6]:
my_sent = df['Body'][1]

In [7]:
ner = getContinuousChunks(my_sent)

In [8]:
company_names = companydf['Name']
vectorizer = TfidfVectorizer(min_df=1)
tf_idf_matrix = vectorizer.fit_transform(company_names)
names_idf_matrix = vectorizer.transform(pd.Series(ner))

In [79]:
result = cosine_similarity(names_idf_matrix,tf_idf_matrix)
result[result<0.8] = 0

In [80]:
mapResults(result, ner, company_names).sort_values('Similarity', ascending = False)

,Left,Right,Similarity
0,SPCE Virgin Galactic,"Virgin Galactic Holdings, Inc.",0.949921
1,SpaceX VG,VG Acquisition Corp.,0.864939
